# Neural Style Transfer

- https://github.com/fchollet/keras/blob/master/examples/neural_style_transfer.py

In [100]:
import time
import numpy as np
from scipy.misc import imsave
from scipy.optimize import fmin_l_bfgs_b
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import vgg19
from keras import backend as K

In [102]:
base_image_path = 'data/blue-moon-lake.jpg'
style_image_path = 'data/starry_night.jpg'
result_prefix = 'results'
iterations = 10
content_weight = 0.025
style_weight = 1.0
tv_weight = 1.0  # total variation

In [7]:
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

In [8]:
img_nrows, img_ncols

(400, 640)

In [15]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [32]:
base_image = K.variable(preprocess_image(base_image_path))  # コンテンツ画像
style_image = K.variable(preprocess_image(style_image_path))  # スタイル画像
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))  # 出力画像（コンテンツとスタイルの結合）
print(base_image)
print(style_image)
print(combination_image)

<tf.Variable 'Variable_9:0' shape=(1, 400, 640, 3) dtype=float32_ref>
<tf.Variable 'Variable_10:0' shape=(1, 400, 640, 3) dtype=float32_ref>
Tensor("Placeholder_3:0", shape=(1, 400, 640, 3), dtype=float32)


In [42]:
# コンテンツ画像、スタイル画像、出力画像を1つのテンソルにまとめる（3枚の画像のバッチになる）
# こうしておくと各画像を入れたときの層の出力がまとめて計算できる！
input_tensor = K.concatenate([base_image, style_image, combination_image], axis=0)
print(input_tensor)

Tensor("concat_3:0", shape=(3, 400, 640, 3), dtype=float32)


In [36]:
model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)

In [37]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [55]:
# layer名 => layer出力の辞書を作成
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [56]:
outputs_dict

{'block1_conv1': <tf.Tensor 'block1_conv1/Relu:0' shape=(3, 400, 640, 64) dtype=float32>,
 'block1_conv2': <tf.Tensor 'block1_conv2/Relu:0' shape=(3, 400, 640, 64) dtype=float32>,
 'block1_pool': <tf.Tensor 'block1_pool/MaxPool:0' shape=(3, 200, 320, 64) dtype=float32>,
 'block2_conv1': <tf.Tensor 'block2_conv1/Relu:0' shape=(3, 200, 320, 128) dtype=float32>,
 'block2_conv2': <tf.Tensor 'block2_conv2/Relu:0' shape=(3, 200, 320, 128) dtype=float32>,
 'block2_pool': <tf.Tensor 'block2_pool/MaxPool:0' shape=(3, 100, 160, 128) dtype=float32>,
 'block3_conv1': <tf.Tensor 'block3_conv1/Relu:0' shape=(3, 100, 160, 256) dtype=float32>,
 'block3_conv2': <tf.Tensor 'block3_conv2/Relu:0' shape=(3, 100, 160, 256) dtype=float32>,
 'block3_conv3': <tf.Tensor 'block3_conv3/Relu:0' shape=(3, 100, 160, 256) dtype=float32>,
 'block3_conv4': <tf.Tensor 'block3_conv4/Relu:0' shape=(3, 100, 160, 256) dtype=float32>,
 'block3_pool': <tf.Tensor 'block3_pool/MaxPool:0' shape=(3, 50, 80, 256) dtype=float32>,
 

In [67]:
def gram_matrix(x):
    assert K.ndim(x) == 3
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def content_loss(base, combination):
    return K.sum(K.square(combination - base))

def style_loss(style, combination):
    assert(K.ndim(style) == 3)
    assert(K.ndim(combination) == 3)
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    # 論文と同じ式
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [78]:
# lossの定義
loss = K.variable(0.)

# 途中のレイヤの出力を得る
# コンテンツ画像、スタイル画像、生成画像をまとめて計算できる
layer_features = outputs_dict['block5_conv2']
print(layer_features)

base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]

# content loss
loss += content_weight * content_loss(base_image_features, combination_features)

# style loss
# style lossは複数のレイヤの出力を使って計算する
feature_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1',
                  'block4_conv1', 'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    print(layer_name, layer_features)
    style_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    s_loss = style_loss(style_features, combination_features)
    loss += (style_weight / len(feature_layers)) * s_loss

# total variation loss
# 生成画像を滑らかにする
loss += tv_weight * total_variation_loss(combination_image)

print(loss)

Tensor("block5_conv2/Relu:0", shape=(3, 25, 40, 512), dtype=float32)
block1_conv1 Tensor("block1_conv1/Relu:0", shape=(3, 400, 640, 64), dtype=float32)
block2_conv1 Tensor("block2_conv1/Relu:0", shape=(3, 200, 320, 128), dtype=float32)
block3_conv1 Tensor("block3_conv1/Relu:0", shape=(3, 100, 160, 256), dtype=float32)
block4_conv1 Tensor("block4_conv1/Relu:0", shape=(3, 50, 80, 512), dtype=float32)
block5_conv1 Tensor("block5_conv1/Relu:0", shape=(3, 25, 40, 512), dtype=float32)
Tensor("add_42:0", shape=(), dtype=float32)


In [79]:
# lossの生成イメージに対する勾配を得る
grads = K.gradients(loss, combination_image)

In [89]:
outputs = [loss]
outputs += grads

In [91]:
outputs  # lossシンボルとgradsシンボルのリスト

[<tf.Tensor 'add_42:0' shape=() dtype=float32>,
 <tf.Tensor 'gradients_1/AddN_16:0' shape=(1, 400, 640, 3) dtype=float32>]

In [95]:
# ノイズ画像（生成画像）を入力して、lossとgradsを返す関数
f_outputs = K.function([combination_image], outputs)
print(f_outputs)

In [109]:
def eval_loss_and_grads(x):
    # xはflat化されているので画像に戻す
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
#     print("###", outs[0], outs[1].shape)
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values

In [110]:
class Evaluator(object):
    
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
    
    def loss(self, x):
        assert self.loss_value is None
        # lossとgradはまとめて計算されるのでloss()を呼び出したときに両方計算しておく
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        # grads()はloss()のあとに呼ばれるのですでにloss()で計算済みのgradを返す
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [111]:
# xは更新対象の生成イメージ
# 生成画像の初期値はノイズ画像ではなくてコンテンツ画像か？
# TODO: ノイズ画像に変えたらどうなる？
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    
    # lossが最小化する値、xが入力、fprimeが勾配
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    
    # 生成画像を保存
    img = deprocess_image(x.copy())
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0


KeyboardInterrupt: 